In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
module_path = os.path.abspath(os.path.join('/data/jiawei_li/doubtbot')) 
if module_path not in sys.path:
    sys.path.append(module_path)
from data import load_data, transform_to_question_answer_pair
from eval import run_critic_eval
from model_wrappers import Llama2Wrapper, Llama3Wrapper

In [2]:
train_data, test_data = load_data()
sample = transform_to_question_answer_pair(train_data[:10])

In [3]:
# critic = Llama2Wrapper("llama2_7b", "meta-llama/Llama-2-7b-chat-hf")
critic = judge = Llama3Wrapper(
    "llama3_8b", "/data/public_models/huggingface/meta-llama/Meta-Llama-3-8B-Instruct"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
run_critic_eval(critic, judge, train_data[:10], "/data/jiawei_li/doubtbot/data/results.json")

  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
 10%|█         | 1/10 [00:09<01:28,  9.82s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 20%|██        | 2/10 [00:17<01:07,  8.46s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Ple

In [3]:
import json
def print_entry(file_path, i=0):
    with open(file_path, 'r') as file:
        data = json.load(file)
        if i < 0 or i >= len(data):
            print(f"Index {i} is out of range for the JSON data.")
            return
        ith_entry = data[i]
        json_str = json.dumps(ith_entry, indent=4)
        for line in json_str.split('\n'):
            print(line)         

In [10]:
def extract_confidence_pairs(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    confidence_pairs = []
    
    for entry in data:
        pre_critique = entry.get("judge_confidence_pre_critique")
        post_critique = entry.get("judge_confidence_post_critique")
        
        if pre_critique is not None and post_critique is not None:
            confidence_pairs.append((pre_critique, post_critique))
    
    return confidence_pairs

In [11]:
extract_confidence_pairs("/data/jiawei_li/doubtbot/data/results.json")

[(0.9956326381262863, 0.2341445588135122),
 (0.9953804753874083, 0.15949336179340529),
 (0.9959349617750168, 0.19929126861241817),
 (0.9983626464931707, 0.6756634646890323),
 (0.9961813111568498, 0.26031967462862315),
 (0.9937442624489288, 0.06039730818017165),
 (0.997176404682894, 0.12338046930955895),
 (0.9943462282442064, 0.3011536833443787),
 (0.9966523141244802, 0.15110926102105712),
 (0.9989210131260612, 0.19367731820795145)]

In [4]:
print_entry("/data/jiawei_li/doubtbot/data/results.json", 1)

{
    "question_answer_pair": {
        "question": "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
        "answer_correct": {
            "numeric": 10.0,
            "proof": "Weng earns 12/60 = $0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $10.\n#### 10"
        },
        "answer_incorrect": {
            "numeric": 50.2,
            "proof": "Weng earns 12/60 = $0.2 per minute.\nWorking 50 minutes, she earned 0.2 + 50 = $50.2.\n#### 50.2"
        }
    },
    "critic": "llama3_8b",
    "judge": "llama3_8b",
    "critique": "The incorrect answer claims that Weng earns $0.2 per minute, which is calculated by dividing the hourly wage of $12 by 60 minutes. This is where the mistake begins. The problem asks how much Weng earned in 50 minutes, not how much she earns per minute.\n\nThe correct calculation would be to multiply the hourly wage by the number of hours Weng worked, which is 50 minutes divided 